In [1]:
%%capture
from pyspark.sql import SparkSession
import glow
spark = SparkSession \
    .builder \
    .appName("PythonPi") \
    .config("spark.jars.packages", "io.projectglow:glow-spark3_2.12:1.1.2") \
    .config("spark.hadoop.io.compression.codecs", "io.projectglow.sql.util.BGZFCodec") \
    .getOrCreate()
spark = glow.register(spark)
import pandas as pd
pd.set_option('max_columns', None)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.projectglow#glow-spark3_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bfa6d9f0-e609-41aa-8ef7-6c90ca875ad1;1.0
	confs: [default]
	found io.projectglow#glow-spark3_2.12;1.1.2 in central
	found org.seqdoop#hadoop-bam;7.9.2 in central
	found com.github.jsr203hadoop#jsr203hadoop;1.0.3 in central
	found org.slf4j#slf4j-api;1.7.25 in central
	found log4j#log4j;1.2.17 in central
	found org.slf4j#slf4j-log4j12;1.7.25 in central
	found org.jdbi#jdbi;2.63.1 in central
	found com.github.broadinstitute#picard;2.21.9 in central
	found com.intel.gkl#gkl;0.8.5 in central
	found commons-io#commons-io;2.4 in central
	found org.broadinstitute#gatk-native-bindings;0.1.0-rc-1 in central
	found org.apache.logging.log4j#log4j-api;2.5 in central
	found org.apache.logging.log4j#log4j-core;2.5 in central
	found com.google.guava#guava;15.0 in central
	

In [2]:
from pyspark.sql.functions import col, regexp_replace, split, regexp_extract, input_file_name
from pyspark.sql.types import LongType, FloatType, DoubleType
from pyspark.sql import DataFrame
import numpy as np
import re

In [3]:
# Define the input file path
input_file = "phenotype.hpoa"

# Read the input file, skipping lines starting with #
lines = spark.sparkContext.textFile(input_file).filter(lambda line: not line.startswith("#"))

# Split the header line to get column names
header = lines.first()
columns = header.split("\t")

# Create a DataFrame by skipping the header and splitting on tabs
data = lines.filter(lambda line: line != header).map(lambda line: line.split("\t"))

# Convert the RDD to a DataFrame
df = spark.createDataFrame(data, columns)

# Save the DataFrame as a Parquet file
df.write.parquet("/sbgenomics/output-files/HPO_2023-10-09/HPO_phenotype.hpoa")

In [7]:
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue=".") \
    .csv("phenotype_to_genes.txt") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/HPO_2023-10-09/HPO_phenotype_to_genes")

In [8]:
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue=".") \
    .csv("genes_to_disease.txt") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/HPO_2023-10-09/HPO_genes_to_disease")

In [9]:
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue=".") \
    .csv("genes_to_phenotype.txt") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/HPO_2023-10-09/HPO_genes_to_phenotype")

In [3]:
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue=".") \
    .csv("/sbgenomics/project-files/HPO_2023-10-09/20231013_hp.tsv") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/HPO_2023-10-09/HPO_ontology")

In [3]:
spark.read.options(inferSchema=True,sep="\t",header=False,nullValue=".") \
    .csv("/sbgenomics/project-files/hg38_MutFormer.txt") \
    .toDF("chromosome", "start", "end", "reference", "alternate", "MutFormer_score") \
    .withColumn('chromosome', regexp_replace('chromosome', 'chr', '')) \
    .withColumn("start",col("start").cast(LongType())) \
    .withColumn("end",col("end").cast(LongType())) \
    .withColumn("MutFormer_score",col("MutFormer_score").cast(DoubleType())) \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/MutFormer")

In [17]:
# greenDB
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue="NA") \
    .csv("GRCh38_GREEN-DB.bed.gz") \
    .withColumn('chromosome', regexp_replace('#Chromosome', 'chr', '')) \
    .withColumn("start",col("Start").cast(LongType())) \
    .withColumn("end",col("End").cast(LongType())) \
    .drop("#Chromosome") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/GREEN-DB")

In [ ]:
# ReMM score version v0.4.0.hg38
# download ReMM file first 
# wget -c https://kircherlab.bihealth.org/download/ReMM/ReMM.v0.4.hg38.tsv.gz
spark.read \
    .option("header", False) \
    .option("sep", "\t") \
    .option("comment", "#") \
    .csv("/sbgenomics/workspace/ReMM.v0.4.hg38.tsv.gz") \
    .toDF("chromosome", "start", "probability") \
    .withColumn("start",col("start").cast(LongType())) \
    .withColumn("probability", col("probability").cast(DoubleType())) \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/ReMM")

In [3]:
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue=".") \
    .csv("/sbgenomics/project-files/gene2refseq_hg38_20231109.tsv.gz") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/gene2refseq_20231109")

In [3]:
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue="-") \
    .csv("gene_info.gz") \
    .where(col('#tax_id') == 9606) \
    .withColumn("alias", split("Synonyms", "\|")) \
    .drop("Synonyms") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/gene_info")

In [6]:
df = spark.read.parquet("/sbgenomics/output-files/gene_info")
df.limit(5).toPandas()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,None,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410...,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20231123,None
1,9606,2,A2M,None,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899...,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20231123,None
2,9606,3,A2MP1,None,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000291190|AllianceGe...,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20231010,None
3,9606,9,NAT1,None,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171...,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20231123,None
4,9606,10,NAT2,None,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156...,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20231123,None


In [16]:
df.where(col('Synonyms').isNotNull()).limit(5).show()

+-------+--------+-------------+-------------+-----------+-------+----------+------------+--------------------+--------------+----------------------------------+-------------------------------------+-------------------+--------------------+-----------------+------------+
|#tax_id|  GeneID|       Symbol|     LocusTag|   Synonyms|dbXrefs|chromosome|map_location|         description|  type_of_gene|Symbol_from_nomenclature_authority|Full_name_from_nomenclature_authority|Nomenclature_status|  Other_designations|Modification_date|Feature_type|
+-------+--------+-------------+-------------+-----------+-------+----------+------------+--------------------+--------------+----------------------------------+-------------------------------------+-------------------+--------------------+-----------------+------------+
|     24|77267466|         dnaA|N5094_RS00005|N5094_00005|   null|      null|        null|chromosomal repli...|protein-coding|                              null|                       